In [ ]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "your_api_key"
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
import os
from collections import defaultdict

def read_and_group_files(prompts_dir):
    grouped_files = defaultdict(list)
    
    for filename in os.listdir(prompts_dir):
        if filename.endswith(".txt"):
            scheme_prefix = filename.split('_')[0]
            grouped_files[scheme_prefix].append(filename)
    
    return grouped_files

In [3]:
def generate_character_description(prompt):
    """
    Function to call GPT-4 API and generate a character description based on the given prompt.
    """
    prompt = (
        f"Generate a very brief character description (such as name, age, gender and occupation) for someone who qualifies for this welfare scheme in the Indian context: {prompt}"
    )

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful assistant who has to generate a character description based on the prompt given. ENSURE THAT THE DESCRIPTION IS COMPLIANT WITH OPENAI'S POLICIES AND DOES NOT CONTAIN OFFENSIVE INFORMATION."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [4]:
# Generate Character Descriptions

def generate_descriptions_for_schemes(prompts_dir):
    """
    Function to generate one character description per welfare scheme by combining all related prompts.
    """
    grouped_files = read_and_group_files(prompts_dir)
    descriptions = {}
    
    for scheme, files in grouped_files.items():
        # Combine all prompts for this scheme
        combined_prompt = ""
        for file in files:
            with open(os.path.join(prompts_dir, file), 'r') as f:
                prompt = f.read().strip()
                combined_prompt += f" {prompt}"
        
        # Generate one character description for the combined prompt
        description = generate_character_description(combined_prompt.strip())
        descriptions[scheme] = description
    
    return descriptions

prompts_dir = "image_prompts"
descriptions = generate_descriptions_for_schemes(prompts_dir)

In [5]:
def save_results(descriptions, output_dir):
    """
    Function to save generated character descriptions directly to output directory.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for scheme, description in descriptions.items():
        output_file = os.path.join(output_dir, f"{scheme}.txt")
        with open(output_file, 'w') as f:
            f.write(description)

output_dir = "image_prompt_context"
save_results(descriptions, output_dir)
print(f"Results saved to {output_dir}")

Results saved to image_prompt_context
